<a href="https://colab.research.google.com/github/kvsaiakhil/Internship-eYSIP-2019/blob/master/IMDb_Senti_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Familiarizing with Pytorch and RNNs for NLP

### "Sentiment analysis of IMDb Reviews"

Reference Link: https://github.com/bentrevett/pytorch-sentiment-analysis

In [0]:
import torch
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.optim as optim

import random

import time



In [0]:
# Seed Initialization for pseudo random sequence generator

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 

In [0]:
#from torchtext import data

TEXT = data.Field(tokenize = 'spacy') # Tokenizing text field (Reviews) 
LABEL = data.LabelField(dtype = torch.float) # Label field is either 'pos'(1) or 'neg'(0)

In [25]:
# from torchtext import datasets

# divide the dataset into test ad train

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

print(vars(train_data.examples[0]))

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['Gung', 'Ho', 'is', 'one', 'of', 'those', 'movies', 'that', 'I', 'never', 'get', 'tired', 'of', 'watching', '.', 'Michael', 'Keaton', 'has', 'always', 'been', 'a', 'favorite', 'of', 'mine', ',', '&', 'he', 'is', 'absolutely', 'hilarious', 'in', 'this', 'movie', '.', 'Matching', 'him', 'step', 'for', 'step', 'is', 'Gedde', 'Watanabe', '.', 'The', 'two', 'of', 'them', 'work', 'wonderfully', 'together', '.', 'Although', 'this', 'movie', 'is', 'a', 'comedy', ',', 'I', 'also', 'like', 'how', 'it', 'shows', 'Hunt', '(', 'Keaton', ')', '&', 'Kazihiro', '(', 'Watanabe', ')', 'struggling', 'in', 'their', 'roles', 'as', 'the', 'leaders', 'of', 'their', 'respective', 'groups', '.', 'They', 'both', 'try', 'so', 'hard', 'to', 'keep', 'the', 'peace', ',', '&', 'then', 'they', 'finally', 'get', 'into', 'a', 'fight', '(', 'which', 'is', 'hysterical', 'to', 'watch', ')', '.', 'First', ',', 'they', "'re", 'both', 'on', 'the',

In [0]:
# divide train data again into train and validation data

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [27]:
# Extracting top 25000 most common words from all reviews

MAX_VOCAB_SIZE = 25_000

# For each review the words, text field contains words with most common words

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.freqs.most_common(20))

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
[('the', 202008), (',', 192080), ('.', 164481), ('a', 109506), ('and', 109236), ('of', 100752), ('to', 93498), ('is', 76004), ('in', 61259), ('I', 54046), ('it', 53369), ('that', 48755), ('"', 44092), ("'s", 43255), ('this', 42291), ('-', 37412), ('/><br', 35659), ('was', 34745), ('as', 30561), ('with', 29885)]


In [0]:
# Initializing Batch Size and selecting device to train the model:  CPU or GPU

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [0]:
#import torch.nn as nn

#Creating the RNN Network : Three layers - Embedding Layer, RNN and Output layer(Fully Connecting Layer)

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [31]:
# Calculating No. of Parameters

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [0]:
# Selecting the optimizer - Stochastic Gradient Descent 

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
#function to calculate accuracy

def binary_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
# Function to train the network

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
# Evaluate loss and accuracy after each epoch
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
#import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
# Training the Network and getting training loss, training accuracy, validation accuracy and validation loss

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 0.694 | Train Acc: 50.11%
	 Val. Loss: 0.698 |  Val. Acc: 49.53%
Epoch: 02 | Epoch Time: 0m 23s
	Train Loss: 0.693 | Train Acc: 49.96%
	 Val. Loss: 0.698 |  Val. Acc: 49.65%
Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 0.693 | Train Acc: 50.17%
	 Val. Loss: 0.698 |  Val. Acc: 50.33%
Epoch: 04 | Epoch Time: 0m 23s
	Train Loss: 0.693 | Train Acc: 49.52%
	 Val. Loss: 0.697 |  Val. Acc: 48.90%
Epoch: 05 | Epoch Time: 0m 23s
	Train Loss: 0.693 | Train Acc: 50.17%
	 Val. Loss: 0.698 |  Val. Acc: 50.38%


In [40]:
# Print text accuracy after training

model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.711 | Test Acc: 46.56%
